**Automatic generation of responses using an LLM**

* **Summary:** This notebook is used for generating the responses: response_yes and response_no given the posts from the user. These responses are needed to train the LLM model using Direct Preference Optimization (DPO) approach. The LLM used is model="gemini-pro" from Google.
* **Datasets:** addiction_2018_features_tfidf_256.csv from https://zenodo.org/records/3941387
* **Contributors:** N Priyanka

In [ ]:
! pip install langchain

! pip install --upgrade --quiet  langchain-google-genai

In [ ]:
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

import pandas as pd

### Fetch Google API Key 
https://makersuite.google.com is the link for generating the API key.

In [ ]:
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

In [ ]:
def detect_condition(question):

    prompt_template = PromptTemplate(
        input_variables = ['question'],
        template = ("""
            Provide me two responses for the question: "response_yes" and "response_no". Use consistent formatting for the labels.
            The response_yes will be the correct evaluation provided as a mental health advisor. 
            The response_no will be an incorrect evaluation of the query.
            While providing response_yes, I want you to act as a mental health adviser. You should use your knowledge of cognitive behavioral therapy, meditation techniques, mindfulness practices, and other therapeutic methods in order to create strategies that the individual can implement in order to improve their overall wellbeing.
            In the response_yes, ask them to contact their nearest helpline. Don't provide contact number since you don't know the location of the user.
        """)
    )

    llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY,temperature=0.2)
    chain = LLMChain(llm=llm, prompt=prompt_template)

    response = chain.run({'question': question})

    return response

### Reading The Metal Health Data

In [ ]:
df = pd.read_csv("./source/mental_health.csv")

df.head()

In [ ]:
posts = pd.DataFrame(df["post"])

# Test the first 10 rows
posts.iloc[0:11]

posts

### Generate Response

In [ ]:
all_response = []
for post in posts:
    response = detect_condition(post)
    all_response.append(response)